In [223]:
import pandas as pd
from datetime import timedelta
from IPython.display import clear_output

df = pd.read_excel('G:\\Pricing\\PPA\\Iberostar\\query1.xlsx') #import the data from the shared drive

In [643]:
df1=df.copy() #create reusable copy
df1.columns #review which columns are within the dataset

Index(['MERCHANTID', 'ORDERID', 'EFFORTID', 'ATTEMPTID', 'MOEA',
       'CURRENCYCODE', 'Payment product ID', 'Payment product name',
       'Acquirer', 'EMAILADDRESS', 'SERVICEPROVIDERID', 'BIN', 'Debit/Credit',
       'Consumer/Commercial', 'Classic/Platinum', 'Bin Country', 'Status ID',
       'Status descriptor', 'Authorized/ Not Authorized', 'Response ID',
       'Response descriptor', 'RTA', 'MES', 'RECEIVEDDATE', 'STATUSDATE',
       'MYAMOUNT', 'COUNT(*)', 'Pre/Post Payment', 'ER', 'Amount in EUR'],
      dtype='object')

In [644]:
#create the key
KEY = df1['MERCHANTID'].astype(str) + df1['CURRENCYCODE'].astype(str) + df1['MYAMOUNT'].astype(str) + df1['EMAILADDRESS'].astype(str)
df1['KEY']=KEY
df1['KEY_SUBGROUP']=0
df1['SUBGROUP_LIMIT']=0
df1['THRESHOLD']=0

In [645]:
grouped_data=[]
unique_keys=df1['KEY'].drop_duplicates().reset_index(drop=True)
grp=df1[['ORDERID','KEY','MYAMOUNT','RECEIVEDDATE','KEY_SUBGROUP','SUBGROUP_LIMIT','THRESHOLD']][(~df1['KEY'].str.contains('nan'))].copy().reset_index(drop=True) #create the data cluster we're going to make our comparisons

In [646]:
for x in range(0,len(unique_keys)):

    grp_dts=grp[(grp['KEY']==unique_keys.iloc[x])] #evaluate the keys
    grp_dts=grp_dts.sort_values(['KEY','RECEIVEDDATE']).reset_index(drop=True).copy() #sort the values by received date and reset the index

    try:
        for y in range(0,len(grp_dts)):
            dt=grp_dts['RECEIVEDDATE'].iloc[0]
            threshold=dt+timedelta(seconds=3600) #this sets the threshold of what cannot be exceeded
            grp_dts['THRESHOLD'][(grp_dts['RECEIVEDDATE']<=threshold)]=threshold #this applies the threshold against everydate
            grp_dts['KEY_SUBGROUP'][(grp_dts['THRESHOLD']!=0)]=y #this applies a grouping    
            grp_dts_upload=grp_dts[(grp_dts['THRESHOLD']!=0)].reset_index(drop=True).copy() #this isolates all instances where there is a threshold 

            for w in range(0,len(grp_dts_upload)):
                grouped_data.append([grp_dts_upload['ORDERID'].iloc[w],grp_dts_upload['KEY'].iloc[w],grp_dts_upload['MYAMOUNT'].iloc[w],grp_dts_upload['RECEIVEDDATE'].iloc[w],grp_dts_upload['THRESHOLD'].iloc[w],grp_dts_upload['KEY_SUBGROUP'].iloc[w]]) #appends the upload to the data array
            
            grp_dts=grp_dts[(grp_dts['THRESHOLD']==0)].reset_index(drop=True).copy()#this then says that the previous parent child relationship has been removed and now the code can be applied to the next
            
            clear_output() #clear iteration status percentage
    
    except:
        for y in range(0,len(grp_dts)):
            dt=grp_dts['RECEIVEDDATE'].iloc[0]
            threshold=dt+timedelta(seconds=3600) #this sets the threshold of what cannot be exceeded
            grp_dts['THRESHOLD'][(grp_dts['RECEIVEDDATE']<=threshold)]=threshold #this applies the threshold against everydate
            grp_dts['KEY_SUBGROUP'][(grp_dts['THRESHOLD']!=0)]=y #this applies a grouping    
            grp_dts_upload=grp_dts[(grp_dts['THRESHOLD']!=0)].reset_index(drop=True).copy() #this isolates all instances where there is a threshold 
            
            for w in range(0,len(grp_dts_upload)):
                grouped_data.append([grp_dts_upload['ORDERID'].iloc[w],grp_dts_upload['KEY'].iloc[w],grp_dts_upload['MYAMOUNT'].iloc[w],grp_dts_upload['RECEIVEDDATE'].iloc[w],grp_dts_upload['THRESHOLD'].iloc[w],grp_dts_upload['KEY_SUBGROUP'].iloc[w]]) #appends the upload to the data array
            
            grp_dts=grp_dts[(grp_dts['THRESHOLD']==0)].reset_index(drop=True).copy()#this then says that the previous parent child relationship has been removed and now the code can be applied to the next
            
            clear_output() #clear iteration status percentage
    
    clear_output()
    print(round((x/len(unique_keys))*100,2)) #show iteration status percentage
            

100.0


In [657]:
#setup the data as a dataframe        
grpd=pd.DataFrame(grouped_data)
#name the columns of the newly created dataframe accordingly
grpd=grpd.rename(columns={0:'ORDERID',1:'KEY',2:'MYAMOUNT',3:'RECEIVEDDATE',4:'THRESHOLD',5:'KEY_SUBGROUP'})

In [659]:
#create a grouped key
grpd['KEY_SUBGROUP']=grpd['KEY_SUBGROUP'].astype(str) #the keygroups are numbers initially this sets them to be strings in order that they be concatenated with the email addresses
grpd['GROUPED_KEY']=grpd['KEY']+grpd['KEY_SUBGROUP'] #concatenation procedure

In [669]:
grpd=grpd[['ORDERID','KEY','MYAMOUNT','RECEIVEDDATE','GROUPED_KEY']].copy() #select on those data elements we want to see
df2=pd.merge(df1,grpd,on=['ORDERID','KEY','MYAMOUNT','RECEIVEDDATE'],how='left') #merge the subgroup key data with the initial data to create a master dataset

In [671]:
#3- Build a function that allows for allow of this to fit together
#create a set of unique values
unique_grouped_keys=df2['GROUPED_KEY'].drop_duplicates().reset_index(drop=True)
data=[]

for x in range(0,len(unique_grouped_keys)):
    grp=df2[(df2['GROUPED_KEY']==unique_grouped_keys.iloc[x])] #evaluate the keys
    grp=grp.sort_values(['KEY','RECEIVEDDATE']).reset_index(drop=True).copy()
    
    #test_min_dt=test_group[['RECEIVEDDATE']].min() #identify the min date
    #test_min_dt=test_min_dt.iloc[0] #convert min date to a series
    #test_min_dt_threshold=test_min_dt+timedelta(seconds=3600) #update the date series to be an hour more than the min
    
    max_date=grp['RECEIVEDDATE'].max()
    grp['FINALTRANSACTION']=max_date
    grp['FINALTRANSACTION_EVALUATION']=grp['RECEIVEDDATE']==grp['FINALTRANSACTION']
    
    for y in range(0,len(grp['RECEIVEDDATE'])):
        data.append([grp['MERCHANTID'].iloc[y],
                     grp['ORDERID'].iloc[y],
                     grp['EFFORTID'].iloc[y],
                     grp['ATTEMPTID'].iloc[y],
                     grp['MOEA'].iloc[y],
                     grp['CURRENCYCODE'].iloc[y],
                     grp['Payment product ID'].iloc[y],
                     grp['Payment product name'].iloc[y],
                     grp['Acquirer'].iloc[y],
                     grp['EMAILADDRESS'].iloc[y],
                     grp['RECEIVEDDATE'].iloc[y],
                     grp['MYAMOUNT'].iloc[y],
                     grp['KEY'].iloc[y],
                     grp['GROUPED_KEY'].iloc[y],
                     grp['FINALTRANSACTION'].iloc[y],
                     grp['FINALTRANSACTION_EVALUATION'].iloc[y]])
    clear_output() #clear iteration status percentage
    print(round((x/len(unique_grouped_keys))*100,2)) #show iteration status percentage

#convert the series data into a dataframe
df3=pd.DataFrame(data)
#name the columns of the newly created dataframe accordingly
df3=df3.rename(columns={0:'MERCHANTID',1:'ORDERID',2:'EFFORTID',3:'ATTEMPTID',4:'MOEA'
                        ,5:'CURRENCYCODE',6:'Payment product ID',7:'Payment product name'
                        ,8:'Acquirer',9:'EMAILADDRESS',10:'RECEIVEDDATE',11:'MYAMOUNT',12:'KEY'
                        ,13:'GROUPED_KEY',14:'FINALTRANSACTION',15:'FINALTRANSACTION_EVALUATION'})
#only bring back those instances where the final transaction evaluation is true
final_trxn=df3[(df3['FINALTRANSACTION_EVALUATION']==True)].reset_index(drop=True)

100.0


In [672]:
#combine the final transaction information to the original data set
output=pd.merge(df2,final_trxn,how='left',on=['MERCHANTID','ORDERID','EFFORTID'
                                             ,'ATTEMPTID','MOEA','CURRENCYCODE'
                                             ,'Payment product ID','Payment product name'
                                             ,'Acquirer','EMAILADDRESS','RECEIVEDDATE','MYAMOUNT'
                                             ,'KEY','GROUPED_KEY'])

In [673]:
output['FINALTRANSACTION_EVALUATION'][(output['FINALTRANSACTION_EVALUATION']!=True)]=False

C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [674]:
output.to_csv('d:\djohnson\Desktop\iberostar_final_auth.csv')